# Liver Model Construction: Notebook

In [1]:
import os
import re
import warnings
from cobra.io import *
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv
import altair as alt
from minspan.minspan import minspan,nnz
# from minspan import minspan,nnz

MASSpy version: 0.1.6


In [2]:
import sys
# from minspan import minspan,nnz
print(sys.version)

3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]


In [3]:
from __future__ import absolute_import, print_function, division
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cobra
import roadrunner

from IPython.display import display, HTML

import sbmlutils
from sbmlutils import dfba
from sbmlutils.dfba.simulator import simulate_dfba
from sbmlutils.dfba import utils
from sbmlutils.dfba import analysis

analysis.set_matplotlib_parameters()

print('sbmlutils:', sbmlutils.__version__)
print('cobra:', cobra.__version__)
print('roadrunner:', roadrunner.__version__)

print(os.getcwd())

ImportError: cannot import name 'dfba' from 'sbmlutils' (C:\Users\sicil\AppData\Roaming\Python\Python38\site-packages\sbmlutils\__init__.py)

In [103]:
maps_dir = os.path.abspath("maps")
data_dir = os.path.abspath("data")
dir = os.path.abspath("")
model_dir = os.path.abspath("models")
minspan_dir= os.path.abspath("minspans_csv")
# extra_dir=os.path.abspath("..")
# minspan_dir

### Getting reactions and metabolite data from RECON 3D

In [104]:
# making dataframe of metabolites

csv_met = os.path.join(data_dir,"gly_met_df.csv")
met_csv_df = pd.read_csv(csv_met,index_col="id")
metabolite_info=met_csv_df.to_dict(orient='index')



# making dataframe of reactions
csv_rxn = os.path.join(data_dir,"reaction_df")
rxn_csv_df = pd.read_csv(csv_rxn,index_col="id") 
reaction_info = rxn_csv_df.to_dict(orient='index')

### Loading cobra model

In [105]:
# Initiate empty model
trial= cobra.Model()

In [106]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = cobra.Metabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    trial.add_metabolites(met)

In [107]:
# Loading reaction data as JSON file to maintain data types as dictionaries 
rxn_json = os.path.join(data_dir,"gly_reaction_df.json")
with open(rxn_json) as json_file:
    rxn_data = json.load(json_file)

In [108]:
for reaction, info in rxn_data.items():
    reaction_obj = cobra.Reaction(reaction)
    reaction_obj.id=reaction
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    trial.add_reaction(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)
    print(reaction)

HEX1
PGI
FBP
PFK
FBA
TPI
GAPD
PGK
PGM
ENO
PYK
LDH_L
ADK1
ATPM


In [109]:
#adding DM_NADH 
# r = cobra.Reaction('NADH_to_ATP')
# trial.add_reaction(r)
# r.build_reaction_from_string('nadh_c+ .5 o2_c + 3.5 h_c + 2.5 pi_c + 2.5 adp_c -> nad_c + 3.5 h2o_c + 2.5 atp_c')

# r = cobra.Reaction('pyr_to_ATP')
# trial.add_reaction(r)
# r.build_reaction_from_string('pyr_c + 3 h2o_c + 12.5 h_c + 12.5 pi_c + 12.5 adp_c -> 3 co2_c + 12.5 h2o_c + 12.5 atp_c + 9 h_c')


# atpm = {'atp_c': -1,'h2o_c': -1,
#     'adp_c': 1,'h_c': 1,'pi_c': 1}
# rxn_2 = cobra.Reaction("ATPM")
# trial.add_reaction(rxn_2)
# rxn.add_metabolites(atpm)



# # add ATPM and NADHM
# atpm = {'atp_c': -1, 'h2o_c': -1, 'adp_c': 1, 'h_c': 1, 'pi_c': 1}
# rxn = cobra.Reaction('ATPM')
# model.add_reaction(rxn)
# rxn.add_metabolites(atpm)


ampm= {'amp_c': -1}
rxn = mass.MassReaction('AMPM')
trial.add_reaction(rxn)
rxn.add_metabolites(ampm)


nadhm = {'nadh_c': -1, 'h_c': 1, 'nad_c': 1}
rxn = cobra.Reaction('NADHM')
trial.add_reaction(rxn)
rxn.add_metabolites(nadhm)

In [110]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {'charge': -1.0}
PGI {}
FBP {'charge': 2.0}
PFK {'charge': -1.0}
FBA {}
TPI {}
GAPD {'charge': -3.0}
PGK {}
PGM {}
ENO {}
PYK {'charge': 1.0}
LDH_L {'charge': -1.0}
ADK1 {}
ATPM {'charge': 1.0}
AMPM {'charge': 2, 'C': -10, 'H': -12, 'N': -5, 'O': -7, 'P': -1}
NADHM {'charge': 1}


In [111]:
trial.metabolites.h_c.charge= 1
# trial.metabolites.h_e.charge= 1
trial.metabolites.pi_c.charge= -2

In [112]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {}
PGI {}
FBP {}
PFK {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PYK {}
LDH_L {}
ADK1 {}
ATPM {}
AMPM {'charge': 2, 'C': -10, 'H': -12, 'N': -5, 'O': -7, 'P': -1}
NADHM {'charge': 2}


In [113]:
for met in ['glc__D_c', 'h2o_c', 'h_c', 'pi_c','lac__L_c',  'amp_c']:
    rxn = cobra.Reaction('EX_%s' % met)
    trial.add_reaction(rxn)
    rxn.add_metabolites({met: -1})
    if met == 'glc__D_c':
        rxn.lower_bound = -1 # by convention negative exchange flux = uptake
    elif met == 'pyr_c':
        rxn.lower_bound = 0
    else:
        rxn.lower_bound = -1000

In [114]:
for reaction in trial.reactions:
    print(reaction)

HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
PGI: g6p_c <=> f6p_c
FBP: fdp_c + h2o_c --> f6p_c + pi_c
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
FBA: fdp_c <=> dhap_c + g3p_c
TPI: dhap_c <=> g3p_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
LDH_L: lac__L_c + nad_c <=> h_c + nadh_c + pyr_c
ADK1: amp_c + atp_c <=> 2.0 adp_c
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
AMPM: amp_c <=> 
NADHM: nadh_c --> h_c + nad_c
EX_glc__D_c: glc__D_c <=> 
EX_h2o_c: h2o_c <=> 
EX_h_c: h_c <=> 
EX_pi_c: pi_c <=> 
EX_lac__L_c: lac__L_c <=> 
EX_amp_c: amp_c <=> 


In [115]:
trial.objective = 'ATPM'
flux_solution = trial.optimize()
flux_solution

,fluxes,reduced_costs
HEX1,1.0,0.0
PGI,1.0,0.0
FBP,0.0,-2.0
PFK,1.0,0.0
FBA,1.0,0.0
...,...,...
EX_h2o_c,0.0,-0.0
EX_h_c,2.0,0.0
EX_pi_c,0.0,0.0
EX_lac__L_c,2.0,0.0


In [116]:
print(trial.reactions.FBA.check_mass_balance())

{}


In [117]:
trial.metabolites.amp_c.summary()

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [118]:
from cobra.io import json
import cobra.test
import os
from os.path import join
from glob import glob



In [119]:

liver_json = os.path.join(model_dir,"gly_json.json")
cobra.io.save_json_model(trial, liver_json )

In [120]:
# import cplex as cplex
# import gurobipy as gurobi

In [121]:
# trial.solver.add("gurobi")
# trial.solver='cplex'
# cobra.util.solver.qp_solvers = ['cplex', 'gurobi']
trial.solver

In [122]:
for model_file in glob(liver_json):
    model_name = model_file.split('/')[-1]
    if 'model' not in model_name:
        continue
    print(model_name)
    model= load_json_model(model_file)
    if 'NADPHM' in model.reactions:
        model.remove_reactions(['NADPHM'])
    # media = ['EX_lac__L_c', 'EX_pyr_c', 'EX_octa_c', 'EX_gln__L_c', 'EX_acetone_c', 'EX_bhb_c',
    #          'EX_glu__L_c', 'EX_ser__L_c', 'EX_cys__L_c', 'EX_gly_c', 'EX_ala__L_c', 'EX_so3_c',
    #         'EX_etoh_c', 'EX_fru_c']
    media = ['EX_glc__D_c']
    for met in media:
        if met in model.reactions:
            model.reactions.get_by_id(met).lower_bound = -1000.

    rxns = [i.id for i in model.reactions]
    blocked = cobra.flux_analysis.find_blocked_reactions(model)
    print(blocked)
    model.remove_reactions(blocked)

    solved_fluxes = minspan(model, cores=3, verbose=False, timelimit=60)
    
    df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in model.reactions])
    df = df/df.abs().max()
    for col in range(len(df.columns)):
        column=df.iloc[:,col]
        vals = column.values
        min_val = min([abs(i) for i in vals if i != 0])
        corr_fac=1/min_val
        # print(corr_fac)
        vals=vals*abs(corr_fac)
        df['Norm'+ str(col)]=vals
    csv_dir = os.path.join(minspan_dir,"gly_csv.csv")
    df.to_csv(csv_dir)

c:\Users\sicil\LiverModel\models\gly_json.json
['ADK1', 'NADHM', 'EX_h2o_c', 'EX_pi_c']
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpfwr3o556.lp
Reading time = 0.00 seconds
: 20 rows, 36 columns, 112 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpk5ukizqe.lp
Reading time = 0.00 seconds
: 56 rows, 54 columns, 215 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpht5gftor.lp
Reading time = 0.00 seconds
: 56 rows, 54 columns, 215 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpoa1kcic1.lp
Reading time = 0.01 seconds
: 56 rows, 54 columns, 215 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpnkjs7em6.lp
Reading time = 0.00 seconds
: 56 rows, 54 columns, 215 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpfd531ql3.lp
Reading time = 0.00 seconds
: 56 rows, 54 columns, 215 nonzeros
Read LP format model from file C:\Users\sicil\Ap

## Making MASS model of glycolysis

In [ ]:
liver= MassModel("Core_Model", array_type='DataFrame', dtype=np.int64)

#### Add the pathway metabolites and their information to a new MASS model

In [ ]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = MassMetabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    liver.add_metabolites(met)

In [ ]:
for reaction, info in rxn_data.items():
    reaction_obj = MassReaction(reaction)
    reaction_obj.id = reaction
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
#     if reaction_obj.id != "DM_nadh" or reaction_obj.id != 'ATPM':
    liver.add_reactions(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)
    print(reaction)

In [ ]:
trial.reactions.HEX1.lower_bound

In [ ]:
liver.reactions.PGK.reverse_stoichiometry(inplace=True)
liver.reactions.PGM.reverse_stoichiometry(inplace=True)
# liver.reactions.PGM

In [ ]:
liver.reactions.PFK

### Getting Standard and Physiological Gibbs energies of reactions

In [ ]:
reaction_str= []

cc = ComponentContribution()

# optional: changing the aqueous environment parameters
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3.31)
cc.ionic_strength = Q_("0.144M")
cc.temperature = Q_("310.15K")
R = 0.00831446261815324 #R = 0.00831446261815324 kJ/mol
from numpy import exp as exp
Keq_df=pd.DataFrame(columns=["Reaction","Keq"])
Keq_df

In [ ]:


for reaction in liver.reactions:
    rxn_name = reaction.id        
    if rxn_name != "ATPM":
        reaction_str = reaction.reaction.split(" <=> ")
        reactants = reaction_str[0]
        reactants = reactants.split(" + ")

        for i in range(len(reactants)):
            string = reactants[i]
            l = len(string)
            reactants[i] = string[:l-2]
            reactants[i]= "bigg.metabolite:" + reactants[i]
        reactants_string= " + ".join(reactants)
        reactants_string = str(reactants_string + " = ")   



        products = reaction_str[1]
        products = products.split(" + ")

        for i in range(len(products)):
            string = products[i]
            l = len(string)
            products[i] = string[:l-2]            
            products[i]= "bigg.metabolite:" + products[i]
        products_string= ' + '.join(products)
        reaction_to_parse = reactants_string + products_string         
        rxn_parsed = cc.parse_reaction_formula(reaction_to_parse)

        print(rxn_name)
        dG0_prime = cc.standard_dg_prime(rxn_parsed)
    #         print(f"ΔG'° = {dG0_prime}")

        dGm_prime = cc.physiological_dg_prime(rxn_parsed)
        dG_prime_value_in_kj_per_mol = dGm_prime.value.m_as("kJ/mol")
        dG0prime_value_in_kj_per_mol = dG0_prime.value.m_as("kJ/mol")
    #     print(f"ΔG'm = {dG_prime_value_in_kj_per_mol}")
        delG = dG_prime_value_in_kj_per_mol
        a = exp( -delG/ (R*310.15)) 
        reaction.Keq = a
        print(f"Keq = {a}")
#         list_keq=[reaction,a]
#         rxn_keq=pd.DataFrame(list_keq)
        df = pd.DataFrame([[reaction,a]], columns=["Reaction","Keq"])
        Keq_df.append(df)
    else:
        reaction.Keq=float("inf")
# Keq_df    

In [ ]:
#adding DM_NADH 
nadhm = {'nadh_c': -1, 'h_c': 1, 'nad_c': 1}
rxn = mass.MassReaction('DM_nadh')
liver.add_reaction(rxn)
rxn.add_metabolites(nadhm)


ampm= {'amp_c': -1}
rxn = mass.MassReaction('AMPM')
liver.add_reaction(rxn)
rxn.add_metabolites(ampm)


In [ ]:
liver.reactions.DM_nadh.Keq=float("inf")

In [ ]:
for met in [
#     'co2_m', 'gdp_m', 'gtp_m', 'pi_m', 'hco3_m','h_m','atp_c','atp_m'
    'glc__D_c','h_c','h2o_c','pi_c','pyr_c','lac__L_c'
#     ,'nadh_c', 'nad_c'


           ]:
    rxn = MassReaction('EX_%s' % met)
    liver.add_reaction(rxn)
    rxn.add_metabolites({met: -1})
    if met == 'glc__D_c':
        rxn.lower_bound = -1 # negative exchange flux = uptake
    elif met == 'pyr_c':
        rxn.lower_bound = 0
    else:
        rxn.lower_bound = -1000

##  Set Fluxes

In [ ]:
reaction_list=[]
for reaction in trial.reactions:
    r_id = reaction.id
    reaction_list.append(r_id)
# reaction_list.append('DM_nadh')
reaction_list

In [ ]:
# making dataframe of reactions

csv_minspan_dir = os.path.join(minspan_dir,"liver_csv_minspan.csv")
minspan_df = pd.read_csv(csv_minspan_dir,index_col="rxn") 

In [ ]:
for reaction, flux in flux_solution[reaction_list].iteritems():
    reaction = liver.reactions.get_by_id(reaction)
    reaction.steady_state_flux = flux * 1278.605 / 1000 # mmol/gdW*hr --> mmol/L * hr ---> mol/L*hr
    print("{0}: {1}".format(reaction.flux_symbol_str,
                            reaction.steady_state_flux))

Setting fluxes from independent fluxes

In [ ]:
def compute_steady_state_fluxes(
    self, dataframe, independent_fluxes, update_reactions=False
):
    r"""Calculate the unique steady state flux for each reaction.

    The unique steady state flux for each reaction in the
    :class:`MassModel` is calculated using defined pathways, independently
    defined fluxes, and steady state concentrations, where index of values
    in the pathways must correspond to the index of the reaction in
    :attr:`MassModel.reactions`.

    Notes
    -----
    The number of individually defined fluxes must be the same as the
    number of pathways in order to determine the solution. For best
    results, the number of pathways to specify must equal the dimension
    of the right nullspace.

    Parameters
    ----------
    pathways : Dataframe of reactions and paths
    array-like
        An array-like object that define the pathways through the reaction
        network of the model. The given pathway vectors must be the same
        length as the number of reactions in the model, with indicies of
        values in the pathway vector corresponding to the indicies of
        reactions in the :attr:`reactions` attribute.
    independent_fluxes : dict
        A ``dict`` of steady state fluxes where :class:`~.MassReaction`\ s
        are keys and fluxes are values to utilize in order to calculate
        all other steady state fluxes. Must be the same length as the
        number of specified pathways.
    update_reactions : bool
        If ``True`` then update the :attr:`.MassReaction.steady_state_flux`
        with the calculated steady state flux value for each reaction.

    Return
    ------
    dict
        A ``dict`` where key:value pairs are the :class:`~.MassReaction`\ s
        with their corresponding calculated steady state fluxes.

    Warnings
    --------
    The indicies of the values in the pathway vector must correspond to the
    indicies of the reactions in the :attr:`reactions` attribute in order
    for the method to work as intended.

    """
    # Check inputs:
    p=[]
    for i in range(len(df.columns)):
        p.append(df[i].values)
    p = np.array(p)
    if not isinstance(p, (np.ndarray, list)):
        raise TypeError(
            "Pathways must be numpy.ndarrays or array-like, "
            "such as a list of lists."
        )


    if len(self.reactions) != p.shape[1]:
        raise ValueError(
            "Pathways must have the same number of columns as"
            " the number of reactions in the model."
        )
    if not isinstance(independent_fluxes, dict):
        raise TypeError("independent_fluxes must be a dict")
    if not isinstance(update_reactions, bool):
        raise TypeError("update_reactions must be a bool")

    coeffs = []
    values = []


    for i, rxn in enumerate(self.reactions):
        if rxn in independent_fluxes:

            values.append(independent_fluxes[rxn])
            
            coeffs.append([path[i] for path in p])
    # Inverse coefficient matrix

    coeffs = np.linalg.inv(coeffs)

    # Obtain the inner product of values and coefficients,
    # then obtain the inner product of the pathways and first inner product
    flux_vector = np.inner(p.T, np.inner(coeffs, values))
    df["flux"]= flux_vector
    
    # Update the reactions if desired
    steady_state_fluxes = {}
    for i, rxn in enumerate(self.reactions):
        steady_state_flux= df.flux[rxn.id]
        steady_state_fluxes.update({rxn: steady_state_flux})
        if update_reactions:
            rxn.steady_state_flux = steady_state_flux

    return steady_state_fluxes

In [ ]:
df = pd.read_csv(csv_dir,index_col=0) 

In [ ]:
liver.metabolites.h_c.charge= 1
# liver.metabolites.h_e.charge= 1
liver.metabolites.pi_c.charge= -2

In [ ]:
#Function to add underscore in front of metabolite identifiers which start with a number
## fix this only after getting Keq from equlibrator because bigg doesn't have _13dpg for example
def prefix_number_id(id_str):
    """Prefix identifiers that start with numbers."""
    if re.match(r"^\d", id_str):
        id_str = "_" + id_str
    return id_str

In [ ]:
#Loop to edit the names using "prefix_number_id" function defined earlier
for metabolite in liver.metabolites:
    new_met_id = prefix_number_id(metabolite.id)
    metabolite.id = new_met_id
liver.repair()

## Setting initial conditions

In [ ]:
import csv
csv_ic = os.path.join(data_dir,"Concentrations_Glycolysis.csv")
# opening the CSV file
with open(csv_ic, mode ='r')as file:
   
  # reading the CSV file
    csvFile = csv.reader(file)

      # displaying the contents of the CSV file
    for name, value in csvFile:
        met_id = liver.metabolites.get_by_id(name)
        met_id.ic = float(value) #M
        print(met_id, ":", met_id.ic)
        print(name,value)

## Set H2O/H Constants

In [ ]:
# Set concentrations of hydrogen, water as fixed
# Assume hydrogen in all compartments are fixed 
for metabolite in [
    "h_c",
#     'h_e','h_m',
#     'h2o_r','h2o_e',
    "h2o_c",
#     "glc__D_e", "pi_e",'pi_m',
    'pi_c'
                  ]:
    metabolite = liver.metabolites.get_by_id(metabolite)
    metabolite.fixed = True

In [ ]:
for metabolite in [
#     'h_m',
"h_c", 
#     'h_e',
"h2o_c",
#     'h2o_r','h2o_e' 
]:
    metabolite = liver.metabolites.get_by_id(metabolite)
    metabolite.ic = 1 #M

In [ ]:
for metabolite in liver.metabolites:
    print("{0}: {1}".format(metabolite.id, metabolite.ic))

In [ ]:
#Formulate QP minimization list for concentrations
conc_solver = ConcSolver(
    liver, 
    excluded_metabolites=[
    "h_c",
#         'h_e', 'h_m',
#     'h2o_r','h2o_e',
        "h2o_c",
#     "pi_e",
#     "glc__D_e", "lac__L_e",
#     "pyr_e"
    ], 
#     excluded_reactions=["Ht", "H2Ot", "H2Oter",'PIter','PIt',
#     'G6Pter','GLCter','GLCt1','PYRt2m','PEPtm',
#     'L_LACt2r','PYRt2'],

    constraint_buffer=1)

conc_solver.setup_feasible_qp_problem(
fixed_conc_bounds=list(liver.fixed))
# fixed_Keq_bounds=liver.reactions.list_attr("Keq_str")
    
#assumption: provide uncertainity to allow for a variance for the Keqs rather than staying fixed
#conc_solver.setup_feasible_qp_problem(Keq_percent_deviation=0.1)

conc_solution = conc_solver.optimize()
conc_solution

In [ ]:
#Function to load data from the excel sheet
def load_data(filepath, sheet_name):
    """Load Liver data from an excel sheet"""
    df = pd.read_excel(engine='openpyxl',
        io=filepath,
        sheet_name=sheet_name,
        index_col=0)
    return df

In [ ]:
#Compare Results
csv_ic = os.path.join(data_dir,"Concentrations_Glycolysis.xlsx")
# ic_df = pd.read_csv(csv_ic)
# ic_df.set_index()
ic_info = load_data(
    filepath=csv_ic,
    sheet_name="Concentrations")

In [ ]:
conc_comparison_fig, ax = plt.subplots(nrows=1, ncols=1,
                                       figsize=(5, 5))

plot_comparison(
    x=ic_info["Concentrations in M"], y=conc_solution,
    compare="concentrations",
    observable=[mid for mid in ic_info.index], ax=ax,
    legend="right outside", plot_function="loglog",
    xlim=(1e-6,0.02), ylim=(1e-6,1e-1),
    xy_line=True,
    xy_legend="best", xlabel="Initial [mol/L]", ylabel="Adjusted [mol/L]")

conc_comparison_fig.tight_layout()
update_model_with_concentration_solution(
    liver, conc_solution, concentrations=True, inplace=True);

In [ ]:
liver.reactions.EX_glc__D_c.Keq = 1
liver.reactions.EX_h_c.Keq = 1 
liver.reactions.EX_h2o_c.Keq = 1 
liver.reactions.EX_pi_c.Keq = 1 
liver.reactions.EX_pyr_c.Keq = 1 
liver.reactions.EX_lac__L_c.Keq = 1 
# liver.reactions.EX_nadh_c.Keq = 1 
# liver.reactions.EX_nad_c.Keq = 1 

In [ ]:
liver.boundary_conditions['glc__D_b']=0.010482807 ##M ## initial condition of the metabolite in the model
liver.boundary_conditions['h2o_b']=1 ##M
liver.boundary_conditions['h_b']=0.0064 ##M
liver.boundary_conditions['pi_b']=0.010482807 ##M
liver.boundary_conditions['pyr_b']=0.00048 ##M
liver.boundary_conditions['nadh_b']=4.63542E-06
liver.boundary_conditions['nad_b']=0.00182531
liver.boundary_conditions['lac__L_b']=0.003261838

In [ ]:
Boundary= [
             "EX_glc__D_c", 
            "EX_h_c", 
            "EX_h2o_c",
            'EX_pyr_c',
            'EX_pi_c',
    "ATPM",
    
#     'EX_nad_c',
    "EX_lac__L_c"
]
for x in Boundary:
    a=liver.reactions.get_by_id(x)
    a.kf=1e6

In [ ]:
liver.calculate_PERCs(fluxes={
                r: v for r, v in liver.steady_state_fluxes.items()
                if not (r.boundary  
#                         or r== liver.reactions.GLCter or r== glycogenolysis.reactions.PIter
#                                or r== glycogenolysis.reactions.G6Pter or r== glycogenolysis.reactions.PIt or 
#                                 r== glycogenolysis.reactions.PYRt2m or 
#                                 r== glycogenolysis.reactions.L_LACt2r or 
#                                 r== glycogenolysis.reactions.PYRt2 or 
#                                 r== glycogenolysis.reactions.PEPtm or r== glycogenolysis.reactions.GLCt1 
#                                 or r== glycogenolysis.reactions.Ht
                       )
},
                      update_reactions=True)

print("Forward Rate Constants\n----------------------")
for reaction in liver.reactions:
    print("{0}: {1:.6f}".format(reaction.kf_str, reaction.kf))

In [ ]:
qcqa_model(liver, parameters=True, concentrations=True,
           fluxes=True, superfluous=True, elemental=True)

In [ ]:
# Setup simulation object
simG=Simulation(liver, verbose=True)
# Simulate from 0 to 1000 with 10001 points in the output
conc_sol, flux_sol = simG.simulate(liver, time=(0, 1e8))
# Quickly render and display time profiles
conc_sol.view_time_profile()

In [ ]:
json_save = os.path.join(model_dir,liver.id + ".json")
json.save_json_model(model=liver, filename=json_save)